In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)

True

In [4]:
reader = PdfReader("data-source/linkedin.pdf")
linkedin = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

#### Tokens and Push Notification function

In [7]:
# Setup APIs
google_api_key = os.getenv('GOOGLE_API_KEY')
google_gai_url="https://generativelanguage.googleapis.com/v1beta/openai/"

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [9]:
def push(message):
    """
    Sends a push notification using the Pushover API.

    This function constructs a payload with a given message, a user key,
    and an application token, then sends it as a POST request to the
    Pushover API endpoint. It prints a success message if the notification
    is sent successfully (HTTP status code 200), otherwise it prints
    an error message including the status code and response text.

    Args:
        message (str): The content of the notification message to be sent.

    Returns:
        None
    """
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    
    response = requests.post(pushover_url, data=payload)
    
    if response.status_code == 200:
        print("✅ Notification sent successfully.")
    else:
        print(f"❌ Failed to send notification: {response.status_code} - {response.text}")

#### Agent and Setup

Tool data storage and schema - structure of a tool response

In [10]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [11]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [16]:
# tool can be a class, function even a plug in. In this code, our tool is a function call
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

Tool Functionalities

In [14]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    """
    Records user details and sends a push notification about the new entry.

    This function takes a user's email, optional name, and optional notes.
    It then sends a push notification (to my window chrome) summarizing
    the recorded interest. It's intended to be used in conjunction with information or a structure
    defined by `record_user_details_json`.

    Args:
        email (str): The email address of the user. This is a mandatory field.
        name (str, optional): The name of the user. Defaults to "Name not provided".
        notes (str, optional): Any additional notes or information about the user's interest.
                               Defaults to "not provided".

    Returns:
        dict: A dictionary indicating the status of the recording.
              Currently returns `{"recorded": "ok"}` regardless of actual storage.
    """
    # use information of record_user_details_json
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [15]:
def record_unknown_question(question):
    """
    Records a user's question that the system was unable to answer and sends a push notification.

    This function takes a question string, sends a push notification
    to alert about the unanswerable question, and indicates that the question has been
    "recorded". It is intended to integrate the structure from `record_unknown_question_json` 
    for storage and logging.

    Args:
        question (str): The specific question that the system could not answer.

    Returns:
        dict: A dictionary indicating the status of the recording.
              Currently returns `{"recorded": "ok"}` regardless of actual storage.
    """
    # use information of record_unknown_question_json
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

#### Tool call Functionalities

In [17]:
def handle_tool_calls(tool_calls):
    """
    Processes a list of tool calls, executing the corresponding functions.

    This function iterates through a list of `tool_call` objects, extracts the
    tool's name and its arguments (assumed to be JSON strings), and then
    dispatches to the appropriate helper function (e.g., `record_user_details`
    or `record_unknown_question`). The results from these tool executions are
    then collected and returned in a structured format suitable for further processing.

    Assumes the existence of functions like `record_user_details` and
    `record_unknown_question` in the current scope.

    Args:
        tool_calls (list): A list of tool call objects, where each object
                           is expected to have a `function` attribute with
                           `name` (str) and `arguments` (JSON string) attributes,
                           and an `id` attribute.

    Returns:
        list: A list of dictionaries, where each dictionary represents the
              result of a tool call. Each dictionary has the following keys:
              - `role` (str): Set to "tool".
              - `content` (str): The JSON-serialized result from the called tool function.
              - `tool_call_id` (str): The ID of the original tool call.
    """
    
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results